# Lecture 11:  Scalar and Vector Fields, Accumulation 

### Sections

* [Introduction](#Introduction)
* [Learning Goals](#Learning-Goals)
* [On Your Own](#On-Your-Own)
    * Scalar Fields
    * Visualizing the Scalar Field by Contours
    * Visualizing the Scalar Field by Value

* [In Class](#In-Class)
    * Use a stream or quiver plot to visualize the gradient
    * Combine them into one plot
    * Compute the divergence of a vector field
    * Relate both of these concepts to flux and accumulation in diffusive problems
* [Homework](#Homework)
* [Summary](#Summary)
* [Looking Ahead](#Looking-Ahead)
* [Reading Assignments and Practice](#Reading-Assignments-and-Practice)

## Introduction
----

In this lecture we will continue building visualization tools to understand scalar and vector quantitites.  Diffusive problems are contextualized in terms of chemical potentials, fluxes and accumulation.  In this lecture we work with scalar fields, compute their vector potentials and develop flux-potential relationships.  

A knowledge of the ideal solution model from chemical thermodynamics will be helpful in completing the lab activities.

[Top of Page](#Sections)

## Learning Goals
----

1.  Know the difference between scalar field and a vector field.
1.  Compute and understand the geometrical interpretation of the gradient.
1.  Develop an understanding for accumulation in diffusive problems.
1.  Continue to practice visualization.

[Top of Page](#Sections)

## On Your Own
----

### Scalar Fields

Let us start by generating a scalar field.  This is a spatially dependant function that generates real numbers.  Many different scalar fields are encountered in science and engineering: temperature, mass/concentration, pressure, etc.  The spatial dependence $c(x,y)$ as an example of a concentration field is one such scalar field.

In [ ]:
%matplotlib notebook

import sympy
from sympy.plotting import plot3d
import numpy as np
import matplotlib.pyplot as plt
x, y = sympy.symbols('x y')

In [ ]:
plot3d(sympy.sin(2*x)*sympy.sin(y), (x, -5, 5), (y, -5, 5));

We use `plot3d()` to visualize this field.  Note that we can represent this as a surface because we are mapping to real numbers.  

Later when we visualize vectors, this will be insufficient.  Your series approximations can also be visualized.  

For example:

In [ ]:
plot3d(sympy.series(sympy.sin(2*x)*sympy.sin(y), x, x0=0, n=7).removeO(), \
       (x, -5, 5), (y, -5, 5));

We will drift between numerical evaluation of functions and symbolic representations of functions.  

Stay alert to the context of symbols as you move along in this lecture.

[Top of Page](#Sections)

### Visualizing the Scalar Field by Contours

We can use the contour plot methods to have a different look at our scalar field - this will be useful when we wish to compare the scalar field to the vector fields that are associated with the scalars.

I find `np.meshgrid` to be very helpful in plotting data.

In [ ]:
%matplotlib notebook
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
delta = 0.025
x = np.arange(-3.0, 3.0, delta)
y = np.arange(-2.0, 2.0, delta)
X, Y = np.meshgrid(x, y)
Z = np.sin(2*X)*np.sin(Y)

In [ ]:
plt.figure()
CS = plt.contour(X, Y, Z)
plt.clabel(CS, inline=1, fontsize=10)
plt.title(r'A Simple Contour Plot of Your Scalar Field')
plt.show;

[Top of Page](#Sections)

### Visualizing the Scalar Field by Value

Not really sure what to call these plots:  heat maps, z-plots, scalar plots, image plots, etc.  Using `np.imshow` we can do the following:

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

In [ ]:
delta = 0.025
x = np.arange(-3.0, 3.0, delta)
y = np.arange(-2.0, 2.0, delta)
X, Y = np.meshgrid(x, y)
Z = np.sin(2*X)*np.sin(Y)

In [ ]:
fig, ax = plt.subplots()
cax = ax.imshow(Z, interpolation='nearest', cmap=cm.coolwarm)
ax.set_title('Scalar Field')
plt.show()

[Top of Page](#Sections)

### DIY:  Using Surface and Contour Plots

Plot the electric field around a point charge.  Use three dimensional surface plots and contour plots to help visualize this potential.

In [ ]:
%matplotlib notebook
import sympy
from sympy.plotting import plot3d
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x, y = sympy.symbols('x y')
plot3d(1/(x**2+y**2), (x, -1, 1), (y, -1, 1))

Create a simple contour plot with labels using default colors.  The inline argument to clabel will control whether the labels are draw over the line segments of the contour, removing the lines beneath the label.

In [ ]:
%matplotlib notebook
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
delta = 0.025
x = np.arange(-2.0, 2.0, delta)
y = np.arange(-2.0, 2.0, delta)
X, Y = np.meshgrid(x, y)
Z = 1/(X**2+Y**2)

In [ ]:
plt.figure(figsize=(4,4))
CS = plt.contour(X, Y, Z, (0.5,1,2,5,10,20,50,100,200))
plt.clabel(CS, inline=1, fontsize=10)
plt.title(r'A Simple Contour Plot of Your Scalar Field')
plt.show;

## In Class
----

### Gradients

Scalar fields have associated vector fields.  One such vector field is the gradient.  The gradient can be thought of as the "spatial rate of change" of a scalar field.  The rate of change depends on the direction that you want to "go" (imagine you are walking along the surface).  Arfken uses the following language to describe the gradient:  "This identifies $\nabla \phi$ as a vector having the direction of the maximum space rate of change of $\phi$ ..."

Let us compute the gradient for our example potential.  You can visualize the gradient operator as a vector with components:

$$\overrightarrow{\nabla} = \frac{\partial}{\partial x} \hat{i}  + \frac{\partial}{\partial y} \hat{j} + \frac{\partial}{\partial z} \hat{k}  $$

and then proceed to perform multiplication as per vector algebra using the dot product.  

There are multiple ways to interact with Python and get at the gradient of a function.  In the first instance we can use the coordinate system capabilities of Sympy so that we can access the built in function gradient().  We start by defining a coordinate system and then calling gradient on our scalar function.  Scalars and vectors are objects of the coordinate system.  [See this page](http://docs.sympy.org/latest/modules/vector/intro.html) for more information on the vector module.

In [ ]:
import sympy
from sympy.vector import CoordSysCartesian
from sympy.vector import gradient

C = CoordSysCartesian('C')
gradient(sympy.sin(2*C.x)*sympy.sin(C.y), C)

[Top of Page](#Sections)

### Streamplots

Note that the above result is a vector - as you would expect.  It is hard however to visualize what this vector looks like.  You'll note that it is also a function of position.  Using the gradient function we get the vector form returned to us.  If we want to visualize the vector field however (Sympy does not provide vector plots yet!) we need to apply our expression in a function at discrete points "by hand".  The next block of code does this.

In [ ]:
%matplotlib notebook

import sympy
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt
import numpy as np

x, y = sympy.symbols('x y')

# This is our function of interest.
def scalar_function():
    return sympy.sin(2*x)*sympy.sin(y)

# This returns a TUPLE that contains the "x" derivative and "y" 
# derivative.  These are the elements of the gradient terms.  We
# manipulate them further below.
def return_gradient_elements(psi):
    u = lambdify((x, y), psi.diff(x), 'numpy')
    v = lambdify((x, y), psi.diff(y), 'numpy')
    return u, v

# This function draws the vectors.  Note that you are passing
# the X, Y positions and the directions u, v.
def plot_streamlines(ax, u, v, xlim=(-1, 1), ylim=(-1, 1)):
    x0, x1 = xlim
    y0, y1 = ylim
    Y, X =  np.ogrid[y0:y1:100j, x0:x1:100j]
    sax = ax.streamplot(X, Y, u(X, Y), v(X, Y), color='cornflowerblue')
    # There are options to streamplot that are useful.  Have a look 
    # at the documentation. One particularly useful feature is the
    # ability to color the streams with a user defined function.
    
def format_axes(ax):
    ax.set_aspect('equal')
    ax.figure.subplots_adjust(bottom=0, top=1, left=0, right=1)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    for spine in ax.spines.itervalues():
        spine.set_visible(False)
        
psi = scalar_function()
u, v = return_gradient_elements(psi)

In [ ]:
xlim = ylim = (-3, 3)
fig, ax = plt.subplots(figsize=(6, 6))
format_axes(ax)
plot_streamlines(ax, u, v, xlim, ylim)

Streamplot in Python has an option to color the streams by a user-defined function. You can compute a function from $u$ and $v$ (for example) and color the streams.  In the absence of colored streams, the plot bears some resemblence to our original filled contour plot, but, it would be nice to see them both together.  You can add plots layer by layer in the same figure.

[Top of Page](#Sections)

### Quiver Plots

In this example we use two different coordinate systems to overlay a quiver plot with a contour plot.

In [ ]:
?np.mgrid

In [ ]:
%matplotlib notebook

import numpy as np
import pylab as plt

x0, x1 = (-5,5)
y0, y1 = (-5,5)

Y, X =  np.mgrid[y0:y1:100j, x0:x1:100j]
Y1, X1 =  np.mgrid[y0:y1:30j, x0:x1:30j]

Z = np.sin(2*X)*np.sin(Y)

# u and v here are the results of applying the gradient operation
# to our scalar field.  Probably wise to check this in a seperate
# code block.
u = (2*np.sin(Y1)*np.cos(2*X1))
v = (np.sin(2*X1)*np.cos(Y1))

In [ ]:
fig, ax = plt.subplots()
plt.contourf(X,Y,Z,10)
plt.quiver(X1,Y1,u,v, color='white')

plt.show()

[Top of Page](#Sections)

### Divergence

The divergence is the application of $\hat{\nabla}$ to another vector.  You can (as above) visualize this as the dot product of the $\hat{\nabla}$ operator and a vector.  What does the divergence of a gradient field return?  A vector or a scalar?

In [ ]:
%matplotlib notebook

import sympy
x, y = sympy.symbols('x y')

from sympy.vector import CoordSysCartesian
from sympy.vector import gradient, divergence

C = CoordSysCartesian('C')
gradientOfOurFunction = gradient(sympy.sin(2*C.x)*sympy.sin(C.y), C)
gradientOfOurFunction

Once again - Sympy comes to the rescue and helps us compute the divergence.

In [ ]:
?divergence

In [ ]:
divergence(gradientOfOurFunction,C)

Note which is a vector field and which is a scalar field.

[Top of Page](#Sections)

### Accumulation

The dot product of $\hat{\nabla}$ and a vector or vector field is a scalar.  This should be recalled from earlier lectures on the properties of matrices and vector algebra.  As expected, this is a scalar quantity.  One physical interpretation is that of the accumulation of matter in a control volume.  The divergence of the flux into (or out of the control volume) is the accumulation.  There are plenty of derivations of this.  Thinking in terms of Fick's Law's of diffusion:

Fick's first law is:

$$\overrightarrow{J} = -M \nabla{\mu} $$

I'm being particular about my choices for notation using $M$ and $\mu$.  Your homework assignment should clarify this.  

Fick's second law is:

$$\frac{\partial C(x,t)}{\partial t} = - \hat{\nabla} \cdot \overrightarrow{J} $$

To be honest - we usually drop the formal over hats and vector notation - we just know it is a vector field.  As indicated below, the sign and magnitude of the accumulation is consistent with F.S.L. and the scalar field above.

### An Example using Sympy

First we set things up for ourselves:

In [ ]:
%matplotlib notebook

import numpy as np
import pylab as plt

import sympy
from sympy.vector import CoordSysCartesian
from sympy.vector import gradient, divergence

We define symbols and our scalar field in the variable `exampleField`.

In [ ]:
x, y = sympy.symbols('x y')
C = CoordSysCartesian('C')

# Define your example scalar field (a concentration like 
# variable C(x,y).
exampleField = -sympy.sin(sympy.pi*C.x)*sympy.cos(sympy.pi*C.y)

We then use the built in `sympy` function `gradient` to compute the gradient:

In [ ]:
# Compute the gradient.
gradientOfField = gradient(exampleField,C)
gradientOfField

We then compute the divergence of the gradient.  Note the absence of `C.i` and `C.j` in the answer indicating that these are not components of a vector.  (Compare this to the last slide.)

In [ ]:
# Compute the divergence.
accumulation = divergence(gradientOfField,C)
accumulation

Take a second here to review the following concepts before moving on:

* scalar field
* vector field
* gradient
* divergence

These are critical concepts to understand more about diffusion and Fick's laws than is usually given in an introductory text.

### Putting it All Together

_Warning:  there is a very high potential for confusion in the following sections if you have not mastered the concepts of scalars, vectors, gradients and divergence.  When folded together in the Python/Sympy representation it can be challenging to see what I'm doing in code.  You may have to go back and review before you move on._

One of the drawbacks to the `vector` submodule in `sympy` is that (it appears) you cannot `lambdify` vector expressions - this means that you have to track vector quantities yourself.  In the next few lines

Here I define an example scalar field.  I'm using the same kind of function as before for the pupose of having something interesting to look at.

In [ ]:
sympyExampleField = sympy.sin(sympy.pi*x)*sympy.cos(sympy.pi*y)

We define the gradient components separately.  I'm keeping track of what points in which direction with the variable names (e.g. `gradFieldX`):

In [ ]:
gradFieldX = sympy.diff(sympyExampleField,x)
gradFieldY = sympy.diff(sympyExampleField,y)

I differentiate each vector component again as dictated by the definition of the divergence and add them together and store that in a different variable `sympyAccumulation`:

In [ ]:
sympyAccumulation = \
-sympy.diff(sympyExampleField,x,2)-sympy.diff(sympyExampleField,y,2)

At this point I have the components of the gradient and the result of the divergence computed.  All that is left to do is to create a version of the sympyAccumulation function that can be used with numpy arrays:

In [ ]:
myAcuumulationFunction = \
sympy.lambdify((x,y), sympyAccumulation, 'numpy')

Two more for the gradients:

In [ ]:
myGradFunctionX = sympy.lambdify((x,y), gradFieldX, 'numpy')
myGradFunctionY = sympy.lambdify((x,y), gradFieldY, 'numpy')

We use our quiver plotting capability to plot:

* the scalar value for the accumulation as `z`
* the $\hat{i}$ component of the gradient as `u`
* the $\hat{j}$ component of the gradient as `v`

In [ ]:
import numpy as np

x0, x1 = (-1,1)
y0, y1 = (-1,1)
plotResolution = 200

Y, X =  np.mgrid[y0:y1:200j, x0:x1:200j]
# Quivers are on a seperate grid since they clutter things up.
Y1, X1 =  np.mgrid[y0:y1:20j, x0:x1:20j]

Z = myAcuumulationFunction(X,Y)
u = myGradFunctionX(X1,Y1)
v = myGradFunctionY(X1,Y1)

In [ ]:
fig, ax = plt.subplots()

plt.contourf(X,Y,Z,10)
plt.colorbar()
plt.quiver(X1,Y1,u,v, color='white')

plt.show()

Examining the above plot we have two pieces of information.  In regions where the chemical potential is small, the accumulation is is high and vice versa.  The vectors tell us in which direction the diffusion of matter will proceed.  Formally, species diffuse down the chemical potential gradient (explaining the minus sign in the expression for Fick's first law as the gradients point "uphill").  

With some additional study you can build a physical intuition for Fick's laws.  This can be generalized to three dimensions quite easily and can be done symbolically with `sympy` and the `vector` submodule, but adds complexity and no additional understanding at this point.

[Top of Page](#Sections)

## Homework
----

1.  (Optional) Using the ideal solution model for the chemical potential, *derive* the accumulation of species in a diffusive problem.

1.  Write a function that returns the accumulation of a potential.  The potential is a `sympy` expression.  The return value should be a sympy expression.  

1. (Optional) Test your function on the chemical potential derived in the first part of the assignment.

[Top of Page](#Sections)

## Summary
----

* The $\hat{\nabla}$ operator can be thought of as a vector that dots into other scalar and vector fields.
* We say that the application of this operator to a scalar field is the gradient.  When applied to a vector field we say that this is the divergence.
* The gradient is a vector field.  The divergence is a scalar field.  Both are "fields" and therefore are spatially dependent.  This is observed by inspection of the results of the gradient and divergence operators on some "toy" functions.
* Using contour plots, heatmaps, and quiver plots we can visualize the gradient and divergence to build physical intuition for differential equations.

[Top of Page](#Sections)

## Looking Ahead
----

* We are moving to Fourier series next lecture.  
* The concept of a generalized inner product of functions will be introduced.
* The idea of an infinite vector space will also be developed.

[Top of Page](#Sections)

## Reading Assignments and Practice
----

* Read the chapter in Arfken on vector analysis.  (Not required, but helpful.)
* Think about what other physical systems are described in terms of gradients.

[Top of Page](#Sections)

## References
----

1.  https://tonysyu.github.io/plotting-streamlines-with-matplotlib-and-sympy.html

[Top of Page](#Sections)